In [1]:
import sys
sys.path.append("../")

import tensorflow as tf
from modules import utils, models

2021-07-19 01:55:10.526117: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# Training Single-Record Models:
## Metadata-Only Model:

In [2]:
metadata_gen = utils.CombinedDataGen(data_file='../data/master_train_df.csv',
                                    out_mode='meta',
                                    mode='train',
                                    shuffle=True,
                                    scaler_dir='../models/scalers')
utils.get_ds_from_gen(metadata_gen,
                      out_mode='meta',
                      mode='train',
                      batch_size=64,
                      cache_dir='')

2021-07-19 01:55:12.801129: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-07-19 01:55:12.882788: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-07-19 01:55:12.882824: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-07-19 01:55:12.885564: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-07-19 01:55:12.888025: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-07-19 01:55:12.888659: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

TypeError: in user code:


    TypeError: tf___image_augment() takes 1 positional argument but 2 were given


In [3]:
metadata_mdl = models.init_meta_model()

2021-07-19 01:53:18.876643: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-07-19 01:53:18.936930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-07-19 01:53:18.936969: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-07-19 01:53:19.093078: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-07-19 01:53:19.192095: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-07-19 01:53:19.344797: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu